<a href="https://colab.research.google.com/github/vinayakShenoy/DL4CV/blob/master/Code/preprocessing/preprocessors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import os
from tensorflow.keras.preprocessing.image import img_to_array
import imutils

In [ ]:
class SimplePreprocessor:
    def __init__(self, width, height, inter=cv2.INTER_AREA):
        self.width = width
        self.height = height
        self.inter = inter
        
    def preprocess(self, image):
        return cv2.resize(image, (self.width, self.height), 
                         interpolation=self.inter)

In [ ]:
class SimpleDatasetLoader:
    def __init__(self, preprocessors=None):
        self.preprocessors = preprocessors
        if self.preprocessors is None:
            self.preprocessors = []
    
    def load(self, imagePaths, verbose=-1):
        data = []
        labels = []
        for (i, imagePath) in enumerate(imagePaths):
            image = cv2.imread(imagePath)
            label = imagePath.split(os.path.sep)[-2]
            if self.preprocessors is not None:
                for p in self.preprocessors:
                    image = p.preprocess(image)
            data.append(image)
            labels.append(label)
            if verbose>0 and i>0 and (i+1)%verbose==0:
                print("[INFO] processed {}/{}".format(i+1, len(imagePaths)))
        return (np.array(data), np.array(labels))

In [ ]:
class ImageToArrayPreprocessor:
    def __init__(self, dataFormat=None):
        self.dataFormat = dataFormat
        
    def preprocess(self, image):
        return img_to_array(image, data_format=self.dataFormat)

In [ ]:
class AspectAwarePreprocessor:
  def __init__(self, width, height, inter=cv2.INTER_AREA):
    self.width = width
    self.height = height
    self.inter = inter
  
  def preprocess(self, image):
    (h, w) = image.shape[:2]
    dW = 0
    dH = 0
    if w<h:
      image = imutils.resize(image, width=self.width, inter=self.inter)
      dH = int((image.shape[0] - self.height)/2.0)
    else:
      image = imutils.resize(image, height=self.height, inter=self.inter)
      dW = int((image.shape[1] - self.width)/2.0)

    (h,w) = image.shape[:2]
    image = image[dH:h-dH, dW]
    return cv2.resize(image, (self.width, self.height), interpolation=self.inter)